In [1]:
import os
sys.path.append(os.path.abspath(".."))
from shopping_agent import run_shopping_agent

In [12]:
# instruction = "لطفاً دراور چهار کشو (کد D14) را برای من تهیه کنید."
# instruction = "عرض پارچه تریکو جودون 1/30 لاکرا گردباف نوریس به رنگ زرد طلایی چقدر است؟"
# instruction = "کمترین قیمت در این پایه برای گیاه طبیعی بلک گلد بنسای نارگل کد ۰۱۰۸ چقدر است؟"
# instruction = "یک لپ تاپ مناسب بهم پیشنهاد کن."
instruction = """
چند تا کالا پیدا کن که برای فروشگاه های متفاوتی هستند و بهم معرفی کن.
"""
result = await run_shopping_agent(instruction)

Step-by-step plan:

1. Understand the user request: The user wants to find multiple products that are available in different shops. The goal is to identify products that have listings in more than one shop.

2. Identify the relevant tables:
   - base_products: to get product information.
   - members: to find shop products linked to base products and their shops.
   - shops: to confirm different shops.

3. Approach:
   - Find base products that have members (shop products) linked to at least two different shops.
   - For these base products, retrieve some basic info like Persian name, English name, and the list of shops offering them.
   - Limit the number of results to a reasonable number (e.g., 10) to avoid overwhelming the user.

4. Query construction:
   - Use SQL to find base products with members linked to multiple distinct shops.
   - Join base_products and members tables.
   - Group by base product and count distinct shops.
   - Filter for count of distinct shops > 1.
   - Retr

In [13]:
result

AgentRunResult(output=ShoppingResponse(message='چند کالا که در فروشگاه\u200cهای مختلف عرضه می\u200cشوند:\n1. شیربرقی لباسشویی الجی sf15(تفکیکی) - در 2 فروشگاه\n2. سرویس قهوه خوری 12 پارچه هتلی سفید تقدیس - در 2 فروشگاه\n3. جارو شارژی روبوراک مدل A20 AIR با زمان کارکرد ۱۸۰ دقیقه - در 4 فروشگاه\n4. قهوه ساز گاستروبک مدل 42711 ظرفیت ۱ لیتر - در 2 فروشگاه\n5. پنکه USB شارک مدل FIBER سایز ۲۵ سانتیمتر - در 2 فروشگاه\n6. گاز کولرگازی مبرد R22 برند دی وای ۱۳/۶ کیلوگرمی - در 7 فروشگاه\n7. ست سرویس بهداشتی هفت پارچه الیکا سفید طلایی - در 2 فروشگاه\n8. گوشت کوب برقی 270 وات تیغه 16 سانت اسمارت - در 3 فروشگاه\n9. جا مایعی بیضی لیمون - در 6 فروشگاه\n10. پیاله ماست خوری گلدکیش مدل کلاسیک کد GK208516 - در 6 فروشگاه', base_random_keys=['aaaavw', 'aaactf', 'aaaivb', 'aaavdk', 'aaavuw', 'aaawnf', 'aababq', 'aabdbg', 'aabrja', 'aabsob'], member_random_keys=None))

In [14]:
dict(result.output)

{'message': 'چند کالا که در فروشگاه\u200cهای مختلف عرضه می\u200cشوند:\n1. شیربرقی لباسشویی الجی sf15(تفکیکی) - در 2 فروشگاه\n2. سرویس قهوه خوری 12 پارچه هتلی سفید تقدیس - در 2 فروشگاه\n3. جارو شارژی روبوراک مدل A20 AIR با زمان کارکرد ۱۸۰ دقیقه - در 4 فروشگاه\n4. قهوه ساز گاستروبک مدل 42711 ظرفیت ۱ لیتر - در 2 فروشگاه\n5. پنکه USB شارک مدل FIBER سایز ۲۵ سانتیمتر - در 2 فروشگاه\n6. گاز کولرگازی مبرد R22 برند دی وای ۱۳/۶ کیلوگرمی - در 7 فروشگاه\n7. ست سرویس بهداشتی هفت پارچه الیکا سفید طلایی - در 2 فروشگاه\n8. گوشت کوب برقی 270 وات تیغه 16 سانت اسمارت - در 3 فروشگاه\n9. جا مایعی بیضی لیمون - در 6 فروشگاه\n10. پیاله ماست خوری گلدکیش مدل کلاسیک کد GK208516 - در 6 فروشگاه',
 'base_random_keys': ['aaaavw',
  'aaactf',
  'aaaivb',
  'aaavdk',
  'aaavuw',
  'aaawnf',
  'aababq',
  'aabdbg',
  'aabrja',
  'aabsob'],
 'member_random_keys': None}

In [15]:
result.usage()

RunUsage(input_tokens=7413, cache_read_tokens=5376, output_tokens=507, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, requests=3, tool_calls=2)

In [16]:
result.all_messages()

[ModelRequest(parts=[SystemPromptPart(content="\n\nYou are an AI Shopping Assistant for Torob. Your job is to answer user instructions by retrieving product information.\n\n\n\nYou have access to the following tools:\n\n1. build_exact_query_and_execute(table_name: str, column_name: str, variable_query: str, limit: int = 10, columns: list[str] | None = None) -> list[RealDictRow] | str:\n   Builds and executes a SQL query that searches for rows where a column exactly matches\n   the given value. You can specify which columns to return. Enforces a minimum limit of 1.\n\n2. build_like_query_and_execute(table_name: str, column_name: str, variable_query: str, limit: int = 10, columns: list[str] | None = None) -> list[RealDictRow] | str:\n   Builds and executes a SQL query that searches for rows where a column contains\n   a given substring (case-insensitive). You can specify which columns to return.\n   Enforces a minimum limit of 3.\n   \n3. generate_sql_query(instruction: str) -> tuple[str